In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
foe = pd.read_csv("FOE.csv")

In [3]:
#Taking a look at our data
foe.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-03-17,0.0,2.722222,2.685185,2.685185,1.450476,47925
1,1980-03-18,0.0,2.703704,2.666667,2.685185,1.450476,77625
2,1980-03-19,0.0,2.703704,2.666667,2.703704,1.460480,35775
3,1980-03-20,0.0,2.777778,2.703704,2.722222,1.470483,42525
4,1980-03-21,0.0,2.777778,2.740741,2.740741,1.480487,37125


In [4]:
#numerical value summary statistics
foe[['Open', 'High', 'Low', 'Close', 'Adj Close']].describe()

,Open,High,Low,Close,Adj Close
count,10590.000000,10590.000000,10590.000000,10590.000000,10590.000000
mean,13.607281,14.631222,14.240872,14.439353,11.952665
std,8.521814,7.381201,7.230212,7.309994,6.597625
min,0.000000,1.010000,0.810000,0.860000,0.860000
25%,7.072500,7.740000,7.420000,7.611111,5.995661
50%,14.860000,15.080000,14.666667,14.865000,12.818146
75%,20.610001,20.847500,20.333332,20.620001,17.534573
max,30.250000,30.937500,30.000000,30.549999,25.350000


In [5]:
foe.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [6]:
#converting the Date into datetime
foe['Date'] = pd.to_datetime(foe.Date, format='%Y-%m-%d')
foe.set_index('Date', inplace=True)

In [9]:
#Five year subset with ontly Date and Close columns
foe_sub = pd.DataFrame(foe[['Adj Close']])
foe_sub = foe_sub.loc[(foe_sub.index >= '2016-01-01')]

In [10]:
foe_sub.head()

,Adj Close
Date,
2016-01-04,10.75
2016-01-05,10.58
2016-01-06,10.05
2016-01-07,9.52
2016-01-08,9.61


In [ ]:
#Average price per month
foe_sub.groupby('year').Close.mean().plot.bar()

In [ ]:
#function to plot a line graph
def line_g(df,x,y,title):
    plt.figure(figsize=(16,8))
    plt.plot(x, y, ".-", data = df)
    plt.title(title, fontsize = 16)
    plt.xlabel("Time", fontsize = 16)
    plt.ylabel("Closing Price", fontsize = 16)
line_g(foe_sub,'Date', 'Close', "Daily Closing Price of FOE")

In [ ]:
monthly = foe_sub.resample('M').mean()
monthly

In [ ]:
line_g(monthly, monthly.index, 'Close', "Average Monthly Closing price of FOE")

In [ ]:
#monthly subset
monthly_sub = monthly.loc[(monthly.index <= '2021-01-01')]
len(monthly_sub)

In [ ]:
monthly_sub.head()

In [ ]:
line_g(monthly_sub, monthly_sub.index, 'Close', "Monthly Closing price of FOE")

In [ ]:
#Decompose the time series to check for seasonality
decomposition_m = sm.tsa.seasonal_decompose(monthly_sub['Close'], model='multiplicative')
fig = decomposition_m.plot()
plt.show()

There is strong evidence of seasonality within the average monthly stock price where there tendency for the price to increase approaching summer and winter. This implies that these are the months the stock performs well which is to be expected as those are when the purchases for sweets are most likely to happen.

In [ ]:
# DF Test using statsmodels adfuller
#this is a test where H0: There is a unit root and the time series is non-stationary / 
#H1: there is no unit root and the data is stationary. 
from statsmodels.tsa.stattools import adfuller

def df_test(time_series):
    
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(time_series, autolag = 'AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)' %key] = value
    
    print (dfoutput)
df_test(monthly_sub.Close)

As the p-value > 0.05, I fail to reject the null hypothesis which means that the data is non-stationary. The next step is to make the data stationary in order to comply with the assumptions of ARIMA. In order to do this, I will use first order differencing.

In [ ]:
# Differencing
monthly_diff = monthly_sub.diff().dropna()
df_test(monthly_diff.Close)

After first order differencing, the ADF test shows that we now have evidence to reject the null hypothesis. In other words, we find no presence of a unit root and therefore the data is stationary. We must now find the best ARIMA parameters.

In [ ]:
#Using auto_arima, the order and seasonal order is found.
from pmdarima import auto_arima

foe_arima = auto_arima(monthly_diff.Close, X=None, start_p=0, d=None, start_q=0, max_p=5, max_d=5, max_q=5, start_P=0, D=None, 
                       start_Q=0, max_P=5, max_D=5, max_Q=5, max_order=5, m=1, seasonal=True, stationary=True, 
                       information_criterion='aic', alpha=0.05, test='kpss', seasonal_test='ocsb', stepwise=False, 
                       n_jobs=10, start_params=None, trend=None, method='lbfgs', maxiter=50, offset_test_args=None, 
                       seasonal_test_args=None, suppress_warnings=True, error_action='trace', trace=False, 
                       random=False, random_state=None, n_fits=10, return_valid_fits=False, out_of_sample_size=0, 
                       scoring='mse', scoring_args=None, with_intercept='auto', sarimax_kwargs=None)

arima_order = foe_arima.order
seasonal_order = foe_arima.seasonal_order

print(foe_arima.summary()) #according to the summary, the best model to use for forecast is a SARIMAX model
print("ARIMA order:", arima_order)
print("Seasonal order:", seasonal_order)

In [ ]:
#Fit the model
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(monthly_diff.Close, order=arima_order, seasonal_order=(0, 0, 0, 12), enforce_stationarity=False, 
                enforce_invertibility=False)
results = model.fit()

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
plt.show()

Looking at the correlogram, time lags of 1 to 10 seems to imply that the monthly average stock price of FOE is stationary and that there seems to be no strong autocorrelation. The closing price also seems to be normally distributed as we can see from both the histogram and the residuals graph. The residuals graph shows no discernable pattern implying that there is no autocorrelation left. I can now use the data for frecasting.

In [ ]:
# forecasting 3 months ahead
forecast = results.get_forecast(steps=3)
forecast_conf_int = forecast.conf_int()

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(monthly_sub.index, monthly_sub['Close'], label='Observed')
plt.plot(forecast.predicted_mean.index, forecast.predicted_mean, color='r', label='Forecast')

# Plot confidence intervals
plt.fill_between(forecast_conf_int.index, forecast_conf_int.iloc[:, 0], forecast_conf_int.iloc[:, 1], color='pink')

plt.xlabel('Time')
plt.ylabel('Closing Price')
plt.legend()
plt.show()